# Import

In [56]:
import numpy as np
import pandas as pd

# Data preporation

In [57]:
# Load previous day data
file_name = 'release/covid19-russia-cases.csv'
rus_df = pd.read_csv(file_name)
rus_df.tail()

,Date,Region/City,Region/City-Eng,Region_ID,Day-Confirmed,Day-Deaths,Day-Recovered,Confirmed,Deaths,Recovered
4917,2020-05-31,Ханты-Мансийский АО,Khanty-Mansiysk,86.0,136.0,0.0,30.0,2436.0,18.0,782.0
4918,2020-05-31,Чеченская Республика,Chechen Republic,95.0,18.0,0.0,3.0,1227.0,13.0,828.0
4919,2020-05-31,Чукотский автономный округ,Chukotka Autonomous Okrug,87.0,2.0,0.0,1.0,80.0,1.0,54.0
4920,2020-05-31,Ямало-Ненецкий АО,Yamalo-Nenets Autonomous Okrug,89.0,65.0,0.0,22.0,2375.0,9.0,975.0
4921,2020-05-31,Ярославская область,Yaroslavl region,76.0,77.0,0.0,5.0,3132.0,14.0,756.0


In [58]:
# Read update data
# Result of data_parsing.ipynb
file_name = 'covid19-russia-cases-upd.csv'
upd_df = pd.read_csv(file_name)
upd_df.sort_values('Confirmed', ascending=False).head()

,Region/City,Confirmed,Date,Deaths,Recovered,Day-Confirmed,Day-Deaths,Day-Recovered
29,Москва,183088.0,2020-06-01,2553.0,82239.0,2297.0,76.0,2060.0
30,Московская область,39723.0,2020-06-01,473.0,8422.0,728.0,24.0,331.0
64,Санкт-Петербург,16313.0,2020-06-01,213.0,5748.0,364.0,9.0,68.0
33,Нижегородская область,10149.0,2020-06-01,104.0,3747.0,315.0,6.0,54.0
67,Свердловская область,5732.0,2020-06-01,29.0,2337.0,283.0,2.0,164.0


In [59]:
# Strip text data
rus_df['Region/City'] = rus_df['Region/City'].astype('str').str.strip('\u200b') 
upd_df['Region/City'] = upd_df['Region/City'].astype('str').str.strip('\u200b')

# Updating

In [60]:
def upd(row):
    reg = row['Region/City']
    
    row['Confirmed'] = rus_df[rus_df['Region/City'] == reg]['Confirmed'].max() + row['Day-Confirmed']
    row['Deaths'] = rus_df[rus_df['Region/City'] == reg]['Deaths'].max() + row['Day-Deaths']
    row['Recovered'] = rus_df[rus_df['Region/City'] == reg]['Recovered'].max() + row['Day-Recovered']
    
    row['Confirmed'] = row['Day-Confirmed'] if np.isnan(row['Confirmed']) else row['Confirmed']
    row['Deaths'] = row['Day-Deaths'] if np.isnan(row['Deaths']) else row['Deaths']
    row['Recovered'] = row['Day-Recovered'] if np.isnan(row['Recovered']) else row['Recovered']
    
    return row.drop('Region/City')

# If cumulative values unknown
# upd_df = upd_df.groupby('Region/City').apply(lambda df: upd(df.iloc[0])).reset_index()

In [61]:
# Date to datetime
upd_df['Date'] = pd.to_datetime(upd_df['Date'])
rus_df['Date'] = pd.to_datetime(rus_df['Date'])

In [62]:
# Region names translation
# from googletrans import Translator
# translator = Translator()

# def translate(rus):
#     eng = translator.translate(rus, src='ru', dest='en')
#     return eng.text

# upd_df['Region/City-Eng'] = upd_df['Region/City'].apply(translate)

In [63]:
# Add region ID and Eng columns
regions_df = pd.read_csv('release/regions-info.csv')

right = regions_df[['Region_ID', 'Region', 'Region_eng']]
upd_df = upd_df.merge(right, left_on='Region/City', right_on='Region', how='left')
upd_df['Region/City-Eng'] = upd_df.Region_eng
upd_df = upd_df[['Date', 'Region/City', 'Region/City-Eng', 'Region_ID', 'Day-Confirmed', 'Day-Deaths', 'Day-Recovered', 'Confirmed', 'Deaths', 'Recovered']]

In [64]:
rus_df = pd.concat([rus_df, upd_df]).reset_index(drop=True)
rus_df.tail(10)

,Date,Region/City,Region/City-Eng,Region_ID,Day-Confirmed,Day-Deaths,Day-Recovered,Confirmed,Deaths,Recovered
4997,2020-06-01,Тюменская область,Tyumen region,72.0,23.0,0.0,6.0,1661.0,8.0,818.0
4998,2020-06-01,Удмуртская Республика,Udmurt republic,18.0,30.0,0.0,1.0,682.0,12.0,412.0
4999,2020-06-01,Ульяновская область,Ulyanovsk region,73.0,101.0,0.0,0.0,2567.0,8.0,713.0
5000,2020-06-01,Хабаровский край,Khabarovsk region,27.0,61.0,1.0,15.0,2142.0,12.0,1017.0
5001,2020-06-01,Ханты-Мансийский АО,Khanty-Mansiysk,86.0,151.0,0.0,27.0,2587.0,18.0,809.0
5002,2020-06-01,Челябинская область,Chelyabinsk region,74.0,254.0,2.0,78.0,3132.0,19.0,1336.0
5003,2020-06-01,Чеченская Республика,Chechen Republic,95.0,20.0,0.0,8.0,1247.0,13.0,836.0
5004,2020-06-01,Чукотский автономный округ,Chukotka Autonomous Okrug,87.0,0.0,0.0,1.0,80.0,1.0,55.0
5005,2020-06-01,Ямало-Ненецкий АО,Yamalo-Nenets Autonomous Okrug,89.0,40.0,1.0,9.0,2415.0,10.0,984.0
5006,2020-06-01,Ярославская область,Yaroslavl region,76.0,78.0,0.0,0.0,3210.0,14.0,756.0


In [65]:
# Negative values fixing
# Negative values can appear if some cases was removed from statistic
def negval_fix(reg):
    last = reg.iloc[-1]
    diff_c = last['Day-Confirmed']*(-1) if last['Day-Confirmed'] < 0 else 0
    diff_d = last['Day-Deaths']*(-1) if last['Day-Deaths'] < 0 else 0
    diff_r = last['Day-Recovered']*(-1) if last['Day-Recovered'] < 0 else 0
    
    last['Day-Confirmed'] += diff_c
    last['Day-Deaths']    += diff_d
    last['Day-Recovered'] += diff_r
    reg.iloc[-1] = last
    
    if diff_c + diff_d + diff_r > 0:
        print(f"Conf {diff_c} Death {diff_d} Rec {diff_r} {last['Region/City']}")
        prev = reg.iloc[-2]
        prev['Confirmed'] -= diff_c
        prev['Deaths']    -= diff_d
        prev['Recovered'] -= diff_r
        reg.iloc[-2] = prev
    
    return reg
    
rus_df = rus_df.groupby('Region/City').apply(negval_fix).reset_index(drop=True)

In [66]:
# Manual fixes if needed
# rus_df.loc[321, 'Confirmed'] = 5
# rus_df.loc[359, 'Day-Confirmed'] = 0


# rus_df.loc[3775, 'Deaths'] = 7
# rus_df.loc[3775, 'Day-Deaths'] = 0

# Data checking

In [67]:
rus_sum = rus_df.loc[rus_df['Region/City'] != 'Diamond Princess']

print('Russia Confirmed:', rus_sum.groupby('Region/City')['Confirmed'].max().sum(),
      rus_sum.groupby('Region/City')['Day-Confirmed'].sum().sum())
print('Russia Deaths:', rus_sum.groupby('Region/City')['Deaths'].max().sum(),
      rus_sum.groupby('Region/City')['Day-Deaths'].sum().sum())
print('Russia Recovered:', rus_sum.groupby('Region/City')['Recovered'].max().sum(),
      rus_sum.groupby('Region/City')['Day-Recovered'].sum().sum())

rus_regs = rus_sum.groupby('Region/City')['Confirmed'].max().reset_index()
rus_regs['Confirmed-ByDay'] = rus_sum.groupby('Region/City')['Day-Confirmed'].sum().reset_index(drop=True)
print(rus_regs.loc[rus_regs['Confirmed'] != rus_regs['Confirmed-ByDay']])

rus_regs = rus_sum.groupby('Region/City')['Recovered'].max().reset_index()
rus_regs['Recovered-ByDay'] = rus_sum.groupby('Region/City')['Day-Recovered'].sum().reset_index(drop=True)
print(rus_regs.loc[rus_regs['Recovered'] != rus_regs['Recovered-ByDay']])

rus_regs = rus_sum.groupby('Region/City')['Deaths'].max().reset_index()
rus_regs['Deaths-ByDay'] = rus_sum.groupby('Region/City')['Day-Deaths'].sum().reset_index(drop=True)
print(rus_regs.loc[rus_regs['Deaths'] != rus_regs['Deaths-ByDay']])

# rus_regs.to_csv('rus_regs.csv')

Russia Confirmed: 415341.0 415341.0
Russia Deaths: 4860.0 4860.0
Russia Recovered: 176087.0 176087.0
Empty DataFrame
Columns: [Region/City, Confirmed, Confirmed-ByDay]
Index: []
Empty DataFrame
Columns: [Region/City, Recovered, Recovered-ByDay]
Index: []
Empty DataFrame
Columns: [Region/City, Deaths, Deaths-ByDay]
Index: []


# Data saving

In [68]:
file_name = 'release/covid19-russia-cases.csv'
rus_df.to_csv(file_name, index=False)